以下內容我們要讓各位可以自行擴充專有領域詞表

使用的資料包含中、英文無監督資料和平行語料，在語料上重新訓練 spm tokenizer，在中文上獲得字詞結合的分詞效果。

以下會執行幾個步驟來進行構建

1. 在大規模語料庫訓練>

- sentencepiece 訓練參考：https://github.com/google/sentencepiece/blob/master/python/sentencepiece_python_module_example.ipynb

2. 擴充既有詞表

- 合併用腳本

3. 比較不同效果



In [1]:
%%bash

pip install --quiet sentencepiece==0.1.99
pip install --quiet datasets

# 步驟ㄧ、準備好語料庫

學習用途使用，這邊我們先使用 Wikipedia 資料集來進行操作 一般而言，你需要使用 wikiextractor 等工具來整理出語料庫，HuggingFace 上其實已經有一整套。

## 為了加速學習我們直接拿中文部分進行操作

如果是在 colab 以外環境，記得自行安裝 git-lfs 套件

In [12]:
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datasets/graelo/wikipedia.git
%cd wikipedia/data/20230601/zh/
!git lfs pull --include "train-*-of-0011.parquet"
%cd ../../../../

Cloning into 'wikipedia'...
remote: Enumerating objects: 2001, done.
remote: Total 2001 (delta 0), reused 0 (delta 0), pack-reused 2001
Receiving objects: 100% (2001/2001), 255.29 KiB | 639.00 KiB/s, done.
Resolving deltas: 100% (615/615), done.
/home/jovyan/自行訓練 LLM 教學/wikipedia/data/20230601/zh
^Cwnloading LFS objects: 100% (22/22), 3.5 GB | 110 MB/s                        

Exiting because of "interrupt" signal.
/home/jovyan/自行訓練 LLM 教學


## 使用整個訓練資料集建立 .txt 數據，用於預先訓練和句子分詞器學習。

In [14]:
%%time
import pandas as pd
from datasets import load_dataset
import csv
import pandas as pd

dataset = load_dataset("wikipedia/data/20230601/zh/", data_files= "train-*-of-0011.parquet",split="train")
dataset.to_csv("wiki.txt", columns=["text"], sep="\t", index=False, header=False, quoting=csv.QUOTE_NONE, escapechar="\\")

Generating train split: 1357881 examples [00:10, 125793.93 examples/s]
Creating CSV from Arrow format: 100%|██████████| 1358/1358 [00:50<00:00, 26.84ba/s]


CPU times: user 55.5 s, sys: 7.57 s, total: 1min 3s
Wall time: 3min 33s


2597793872

# 步驟二、開始訓練

這段程式碼會因為記憶體不足而導致錯誤，因此您應該透過設定 --input_sentence_size=<size> --shuffle_input_sentence=true 來限制輸入資料（對於具有 12GB 記憶體的 Colab 實例，約為 --input_sentence_size=12000000 ），請在記憶體充足的環境下運作。好像58GB就可以了。
    
https://arc.net/l/quote/ttjpakta

In [ ]:
import sentencepiece as spm
import os

os.makedirs("zh_wiki_bpe_model_sp",exist_ok=True)


spm.SentencePieceTrainer.train(
    input='wiki.txt',
    model_prefix='zh_wiki_bpe_model_sp/zh_wiki_bpe_hf',
    user_defined_symbols=["，","。","：","？","（","）","「","」"],
    max_sentence_length=2048,
    split_digits=True,
    model_type="bpe",
    byte_fallback=True,
    train_extremely_large_corpus=True,
    vocab_size=50000,
    split_by_unicode_script=True,
    allow_whitespace_only_pieces=True,
    remove_extra_whitespaces=False,
    normalization_rule_name="nmt_nfkc_cf",
    num_threads=48,
    input_sentence_size=12000000,
    shuffle_input_sentence=True
)

## 使用訓練好的分詞器進行推理

In [ ]:
# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
model_file = 'zh_wiki_bpe_model_sp/zh_wiki_bpe_hf' + '.model'
sp.load(model_file)

# encode: text => id
print(sp.encode_as_pieces('潜伏性感染又称潜在性感染。慕容复来到河边,this is a test'))
print(sp.encode_as_ids('this is a test'))

# decode: id => text
print(sp.decode_pieces(['▁This', '▁is', '▁a', '▁t', 'est']))
# print(sp.decode_ids([209, 31, 9, 375, 586]))

## 轉換成 HF 格式

In [ ]:
from transformers import LlamaTokenizerFast

chinese_sp_model_file = "zh_wiki_bpe_model_sp/zh_wiki_bpe.model"

# LlamaTokenizer can use spm model file directly
tokenizer = LlamaTokenizerFast(vocab_file=chinese_sp_model_file)
tokenizer.save_pretrained("zh_wiki_bpe_model_hf")
print("done")

# 步驟三、跟其他模型 Tokenizer 進行比較

In [ ]:
from transformers import AutoTokenizer,PreTrainedTokenizer,LlamaTokenizer
from tokenizers import Tokenizer,AddedToken
from tokenizers.models import WordLevel

In [ ]:
#
zh_wiki_bpe_tokenizer = AutoTokenizer.from_pretrained("zh_wiki_bpe_model_hf")
taiwan_llama_tokenizer = AutoTokenizer.from_pretrained("yentinglin/Taiwan-LLM-7B-v2.1-chat")
breeze_mistral_tokenizer = AutoTokenizer.from_pretrained("MediaTek-Research/Breeze-7B-Instruct-v0_1")
bailong_mistral_tokenizer = AutoTokenizer.from_pretrained("INX-TEXT/Bailong-instruct-7B")

In [ ]:
text = """
    尚-雅克·盧梭（法語：Jean-Jacques Rousseau，法語發音：[ʒɑ̃ ʒak ʁuso]；1712年6月28日—1778年7月2日）是啟蒙時代的法國與日內瓦哲學家、政治理論家、文學家和音樂家。
    盧梭的小說作品《愛彌兒》（Émile）是一篇關於全人公民教育的哲學論文，對康德影響甚大。其言情小說《新愛洛伊斯》對前浪漫主義（pre-romanticism）[19]及浪漫主義時期的小說發展十分重要[20]。
    不過，一些知名學者認為盧梭雖然預示了浪漫主義的誕生，但是其「現代文學姿態」其實早已「超越了感傷的浪漫主義」，而其嶄新的語言觀甚至「一直延續到了超現實主義那裡」[21]。
    """

In [ ]:
print("zh-wiki bpe tokeinzer")
print(token_str:=convert_to_token(zh_wiki_bpe_tokenizer,text),f"\ntoken_count:{len(token_str.split())}")

print()
print("Taiwan LLM tokeinzer")
print(token_str:=convert_to_token(taiwan_llama_tokenizer,text),f"\ntoken_count:{len(token_str.split())}")
print()

print("Breeze")
print(token_str:=convert_to_token(breeze_mistral_tokenizer,text),f"\ntoken_count:{len(token_str.split())}")
print()

print("Bailong")
print(token_str:=convert_to_token(bailong_mistral_tokenizer,text),f"\ntoken_count:{len(token_str.split())}")
print()